In [19]:
!pip install avro pandas pandasql
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/avro/train.avro
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/avro/train.avsc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100   369  100   369    0     0    492      0 --:--:-- --:--:-- --:--:--   492
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100   271  100   271    0     0    736      0 --:--:-- --:--:-- --:--:--   736


In [2]:
from avro.io import DatumReader
from avro.datafile import DataFileReader

import json, copy

def print_avro(avro_file, max_record_num=None):
    if max_record_num is not None and max_record_num <= 0:
        return

    with open(avro_file, 'rb') as avro_handler:
        reader = DataFileReader(avro_handler, DatumReader())
        record_count = 0
        for record in reader:
            record_count = record_count+1
            print(record)
            if max_record_num is not None and record_count == max_record_num:
               break

print_avro(avro_file='train.avro')

{'features': [0, 0, 0, 1, 4], 'label': None, 'dataType': 'TRAINING'}
{'features': [0, 0], 'label': 2, 'dataType': 'TRAINING'}
{'features': [0], 'label': 3, 'dataType': 'VALIDATION'}
{'features': [1], 'label': 4, 'dataType': 'VALIDATION'}


In [3]:
def save_and_print_schema(avro_file, schema_file):
    with open(avro_file, 'rb') as avro_handler:
        reader = DataFileReader(avro_handler, DatumReader())
        metadata = copy.deepcopy(reader.meta)
        schema_from_file = json.loads(metadata['avro.schema'])

    with open(schema_file, "w") as schema_handler:
        json.dump(schema_from_file, schema_handler, indent=4)
        schema_handler.close()
        print(json.dumps(schema_from_file, indent=4))

save_and_print_schema("train.avro", "train_schema.avsc")

{
    "type": "record",
    "name": "ImageDataset",
    "fields": [
        {
            "type": {
                "type": "array",
                "items": "int"
            },
            "name": "features"
        },
        {
            "type": [
                "int",
                "null"
            ],
            "name": "label"
        },
        {
            "type": {
                "type": "enum",
                "name": "dataTypes",
                "symbols": [
                    "TRAINING",
                    "VALIDATION"
                ]
            },
            "name": "dataType"
        }
    ]
}


In [4]:
import pandas as pd

def import_avro(avro_file, max_record_num=None):
    ret_list=[]
    if max_record_num is not None and max_record_num <= 0:
        return

    with open(avro_file, 'rb') as avro_handler:
        reader = DataFileReader(avro_handler, DatumReader())
        record_count = 0
        for record in reader:
            record_count = record_count+1
            ret_list.append(record)
            if max_record_num is not None and record_count == max_record_num:
               break
        return(ret_list)

ret_list = import_avro("train.avro")
ret_frame = pd.DataFrame(ret_list)
ret_frame

,features,label,dataType
0,"[0, 0, 0, 1, 4]",NaN,TRAINING
1,"[0, 0]",2.0,TRAINING
2,[0],3.0,VALIDATION
3,[1],4.0,VALIDATION


In [8]:
from pandasql import sqldf

pd2 = ret_frame.drop("features", axis="columns") # pansasql does not support lists in columns
pd2

,label,dataType
0,NaN,TRAINING
1,2.0,TRAINING
2,3.0,VALIDATION
3,4.0,VALIDATION


In [9]:
sqldf('''SELECT DISTINCT dataType FROM pd2''')

,dataType
0,TRAINING
1,VALIDATION


In [10]:
sqldf('''SELECT datatype, count(dataType) FROM pd2 GROUP BY dataType''')

,dataType,count(dataType)
0,TRAINING,2
1,VALIDATION,2
